# Hyperwave Local Workflow

This notebook demonstrates the **local workflow** for FDTD photonics simulations.
All CPU steps run locally using the hyperwave functions directly. Only the GPU 
simulation step requires an API call.

**Local Steps (run on your machine):**
1. Load GDSFactory component → theta pattern
2. Apply density filtering
3. Build 3D layer structure
4. Add absorbing boundaries
5. Create mode source
6. Set up monitors

**API Step (uses credits):**
7. Run FDTD simulation on cloud GPU

**Analysis (local):**
8. Analyze results

## Installation

In [ ]:
%pip install git+https://github.com/spinsphotonics/hyperwave-community.git --force-reinstall -q

## Imports

In [ ]:
import hyperwave_community as hwc
import gdsfactory as gf
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp

---
## Device Setup (Local)

These steps run entirely on your local machine.

### Step 1: Load GDSFactory Component

Load a photonic component from GDSFactory and convert it to a binary theta pattern.

In [ ]:
# Component settings
COMPONENT_NAME = "mmi2x2_with_sbend"
RESOLUTION_UM = 0.02  # 20nm resolution

# Load component from GDSFactory
component = gf.components.mmi2x2_with_sbend()

# Convert to theta pattern
theta, device_info = hwc.component_to_theta(
    component=component,
    resolution=RESOLUTION_UM,
)

print(f"Theta shape: {theta.shape}")
print(f"Device size: {device_info['physical_size_um']} um")

# Visualize
plt.figure(figsize=(12, 4))
plt.imshow(theta, cmap='gray', aspect='equal')
plt.title(f'{COMPONENT_NAME} - Theta Pattern')
plt.xlabel('x (pixels)')
plt.ylabel('y (pixels)')
plt.colorbar(label='Material (0=clad, 1=core)')
plt.show()

### Step 2: Apply Density Filtering

Smooth the theta pattern with density filtering. This helps with numerical stability.

In [ ]:
# Material properties
N_CORE = 3.48      # Silicon refractive index
N_CLAD = 1.4457    # SiO2 cladding refractive index

# Padding for absorbers and monitors
PADDING = (100, 100, 0, 0)  # (left, right, top, bottom) in pixels

# Apply density filtering
density_core = hwc.density(
    theta=theta,
    pad_width=PADDING,
    radius=3,  # Smoothing radius
)

density_clad = hwc.density(
    theta=jnp.zeros_like(theta),
    pad_width=PADDING,
    radius=3,
)

print(f"Density shape (with padding): {density_core.shape}")

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
ax1.imshow(density_core, cmap='PuOr')
ax1.set_title('Density Core (waveguide)')
ax2.imshow(density_clad, cmap='PuOr')
ax2.set_title('Density Clad (cladding only)')
plt.tight_layout()
plt.show()

### Step 3: Build 3D Layer Structure

Stack layers to create the 3D permittivity structure.

In [ ]:
# Layer dimensions
WG_HEIGHT_UM = 0.22         # Waveguide height
TOTAL_HEIGHT_UM = 4.0       # Total simulation height
VERTICAL_RADIUS = 2.0       # Vertical blur radius

# Calculate layer thicknesses in cells
wg_thickness_cells = int(np.round(WG_HEIGHT_UM / RESOLUTION_UM))
clad_thickness_cells = int(np.round((TOTAL_HEIGHT_UM - WG_HEIGHT_UM) / 2 / RESOLUTION_UM))

print(f"Waveguide thickness: {wg_thickness_cells} cells")
print(f"Cladding thickness: {clad_thickness_cells} cells")

# Define layers
waveguide_layer = hwc.Layer(
    density_pattern=density_core,
    permittivity_values=(N_CLAD**2, N_CORE**2),  # (clad, core)
    layer_thickness=wg_thickness_cells,
)

cladding_layer = hwc.Layer(
    density_pattern=density_clad,
    permittivity_values=N_CLAD**2,
    layer_thickness=clad_thickness_cells,
)

# Create 3D structure
structure = hwc.create_structure(
    layers=[cladding_layer, waveguide_layer, cladding_layer],
    vertical_radius=VERTICAL_RADIUS,
)

_, Lx, Ly, Lz = structure.permittivity.shape
print(f"Structure dimensions: ({Lx}, {Ly}, {Lz})")

# Visualize cross-sections
structure.view(
    show_permittivity=True,
    show_conductivity=False,
    axis="z",
    position=Lz // 2,
    cmap_permittivity="viridis",
)

structure.view(
    show_permittivity=True,
    show_conductivity=False,
    axis="x",
    position=Lx - 50,
    cmap_permittivity="viridis",
)

### Step 4: Add Absorbing Boundaries

Add PML absorbing boundaries to prevent reflections.

In [ ]:
# Absorber parameters (optimized for 20nm resolution)
ABS_WIDTH_X = 82
ABS_WIDTH_Y = 41
ABS_WIDTH_Z = 41
ABS_COEFF = 6.17e-4

abs_shape = (ABS_WIDTH_X, ABS_WIDTH_Y, ABS_WIDTH_Z)

# Create absorption mask
absorber = hwc.create_absorption_mask(
    grid_shape=(Lx, Ly, Lz),
    absorption_widths=abs_shape,
    absorption_coeff=ABS_COEFF,
    show_plots=True,
)

# Add absorber to structure conductivity
structure.conductivity = jnp.zeros_like(structure.conductivity) + absorber

# Visualize conductivity (absorber regions)
structure.view(
    show_permittivity=False,
    show_conductivity=True,
    axis="z",
    position=Lz // 2,
)

### Step 5: Create Mode Source

Solve for the fundamental waveguide mode at the input port.

In [ ]:
# Wavelength and frequency settings
WL_UM = 1.55  # Wavelength in microns
wl_cells = WL_UM / RESOLUTION_UM
freq_band = (2 * jnp.pi / wl_cells, 2 * jnp.pi / wl_cells, 1)

# Source position (after absorber region)
source_pos_x = ABS_WIDTH_X + 5

# Find waveguide center for mode solving
# For MMI with s-bends, bottom waveguide is in lower half
y_center = Ly // 4  # Approximate center of bottom input waveguide

# Create mode source
source_field, source_offset, mode_info = hwc.create_mode_source(
    structure=structure,
    freq_band=freq_band,
    mode_num=0,  # Fundamental mode
    propagation_axis="x",
    source_position=source_pos_x,
    perpendicular_bounds=(0, Ly // 2),  # Bottom half only for input waveguide
    visualize=True,
    visualize_permittivity=True,
)

print(f"Source field shape: {source_field.shape}")
print(f"Source offset: {source_offset}")

### Step 6: Set Up Monitors

Configure field monitors at input and output ports.

In [ ]:
# Create monitor set
monitors = hwc.MonitorSet()

# Input monitor (after source)
monitors.add_monitors_at_position(
    structure=structure,
    axis="x",
    position=ABS_WIDTH_X + 10,
    label="Input",
)

# Output monitors (before right absorber)
monitors.add_monitors_at_position(
    structure=structure,
    axis="x",
    position=Lx - (ABS_WIDTH_X + 10),
    label="Output",
)

# List all monitors
monitors.list_monitors()

# Visualize monitor positions
monitors.view(
    structure=structure,
    axis="z",
    position=Lz // 2,
    source_position=source_pos_x,
    absorber_boundary=absorber,
)

---
## GPU Simulation (API)

This step runs on cloud GPU and requires an API key.

In [ ]:
# Configure API key (only needed for GPU simulation)
API_KEY = "YOUR_API_KEY_HERE"

# Run FDTD simulation on cloud GPU
results = hwc.simulate(
    structure=structure,
    source_field=source_field,
    source_offset=source_offset,
    freq_band=freq_band,
    monitors=monitors,
    mode_info=mode_info,
    simulation_steps=20000,
    check_every_n=1000,
    source_ramp_periods=5.0,
    add_absorption=True,
    absorption_widths=abs_shape,
    absorption_coeff=ABS_COEFF,
    api_key=API_KEY,
    gpu_type="B200",  # Options: "B200", "H200", "H100", "A100", etc.
)

print(f"GPU time: {results['sim_time']:.2f}s")
print(f"Performance: {results['performance']:.2e} grid-points×steps/s")

In [ ]:
# Quick visualization of all monitors
hwc.quick_view_monitors(results, component="all")

---
## Result Analysis (Local)

Analyze transmission and visualize fields locally.

In [ ]:
# Get monitor data
monitor_data = results['monitor_data']

# Helper function for Poynting vector calculation
def S_from_slice(field_slice):
    """Calculate Poynting vector from field slice."""
    E = field_slice[:, :3, :, :]  # Ex, Ey, Ez
    H = field_slice[:, 3:, :, :]  # Hx, Hy, Hz
    
    S = jnp.zeros_like(E, dtype=jnp.float32)
    S = S.at[:, 0, :, :].set(jnp.real(E[:, 1] * jnp.conj(H[:, 2]) - E[:, 2] * jnp.conj(H[:, 1])))
    S = S.at[:, 1, :, :].set(jnp.real(E[:, 2] * jnp.conj(H[:, 0]) - E[:, 0] * jnp.conj(H[:, 2])))
    S = S.at[:, 2, :, :].set(jnp.real(E[:, 0] * jnp.conj(H[:, 1]) - E[:, 1] * jnp.conj(H[:, 0])))
    
    return S * 0.5

# Get field data from monitors
input_fields = monitor_data['Input_bottom']
output_bottom = monitor_data['Output_bottom']
output_top = monitor_data['Output_top']

print(f"Monitor field shape: {input_fields.shape}")

# Average across monitor thickness (X dimension)
input_plane = jnp.mean(input_fields, axis=2)
out_bottom_plane = jnp.mean(output_bottom, axis=2)
out_top_plane = jnp.mean(output_top, axis=2)

# Calculate Poynting vectors
S_in = S_from_slice(input_plane)
S_out_bottom = S_from_slice(out_bottom_plane)
S_out_top = S_from_slice(out_top_plane)

# Calculate power (X-component)
power_in = jnp.abs(jnp.sum(S_in[:, 0, :, :], axis=(1, 2)))
power_out_bottom = jnp.abs(jnp.sum(S_out_bottom[:, 0, :, :], axis=(1, 2)))
power_out_top = jnp.abs(jnp.sum(S_out_top[:, 0, :, :], axis=(1, 2)))
power_out_total = power_out_bottom + power_out_top

# Calculate metrics
total_transmission = power_out_total / power_in
split_bottom = power_out_bottom / power_out_total
split_top = power_out_top / power_out_total

# Print results
print("\n" + "=" * 60)
print("TRANSMISSION ANALYSIS")
print("=" * 60)
print(f"Input Power:      {float(power_in[0]):.4e}")
print(f"Output Bottom:    {float(power_out_bottom[0]):.4e}")
print(f"Output Top:       {float(power_out_top[0]):.4e}")
print(f"Total Output:     {float(power_out_total[0]):.4e}")
print("-" * 60)
print(f"Transmission:     {float(total_transmission[0]):.1%}")
print(f"Split Ratio:      {float(split_bottom[0]):.1%} / {float(split_top[0]):.1%}")
print("=" * 60)

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

wavelengths = [WL_UM]

# Power levels
axes[0].bar(['Input', 'Out Bottom', 'Out Top'], 
            [float(power_in[0]), float(power_out_bottom[0]), float(power_out_top[0])])
axes[0].set_ylabel('Power (a.u.)')
axes[0].set_title('Power Flow')
axes[0].grid(True, alpha=0.3)

# Transmission
axes[1].bar(['Transmission'], [float(total_transmission[0]) * 100])
axes[1].axhline(y=100, color='gray', linestyle='--', alpha=0.5)
axes[1].set_ylabel('Transmission (%)')
axes[1].set_ylim(0, 110)
axes[1].set_title('Total Transmission')
axes[1].grid(True, alpha=0.3)

# Split ratio
axes[2].bar(['Bottom', 'Top'], 
            [float(split_bottom[0]) * 100, float(split_top[0]) * 100])
axes[2].axhline(y=50, color='gray', linestyle='--', alpha=0.5)
axes[2].set_ylabel('Split (%)')
axes[2].set_ylim(0, 100)
axes[2].set_title('Output Power Distribution')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTransmission: {float(total_transmission[0]):.1%}")
print(f"Split: {float(split_bottom[0]):.1%} bottom / {float(split_top[0]):.1%} top")

---
## Summary

| Step | Function | Runs On | Cost |
|------|----------|---------|------|
| 1 | `component_to_theta()` | Local | Free |
| 2 | `density()` | Local | Free |
| 3 | `Layer()`, `create_structure()` | Local | Free |
| 4 | `create_absorption_mask()` | Local | Free |
| 5 | `create_mode_source()` | Local | Free |
| 6 | `MonitorSet()` | Local | Free |
| 7 | `simulate()` | Cloud GPU | Credits |
| 8 | Analysis | Local | Free |

### When to Use Local Workflow

- **Custom structures**: Create theta patterns that aren't in GDSFactory
- **Inverse design**: Optimize theta as a design variable
- **Debugging**: Inspect all intermediate arrays
- **Large structures**: Pre-process locally before committing to GPU time